In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [2]:
df=pd.read_csv(r"Data Engineer Assignment.csv")

In [3]:
df['eventtime'] = df['event_time'].apply(lambda x : datetime.strptime(x[:-7],'%Y-%m-%d %H:%M:%S'))

In [4]:
df['date'] = df["event_time"].apply(lambda x:datetime.date(datetime.strptime(x[:-7],'%Y-%m-%d %H:%M:%S')))

In [5]:
df['time'] = df['event_time'].apply(lambda x : datetime.time(datetime.strptime(x[:-7],'%Y-%m-%d %H:%M:%S')))

In [6]:
df1=df[df.time <=datetime.time(datetime.strptime('1900-01-01 19:00:00','%Y-%m-%d %H:%M:%S'))]

In [7]:
df1['offline']=df1["detail"].str.contains('False')


C:\Users\Vidya Sagar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
df1['online'] = (df1['detail'].str.contains('True') | df1['source'].str.contains('Action on Job'))


C:\Users\Vidya Sagar\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df1=df1.drop(df1[(df1.offline==False) & (df1.online==False)].index)

In [10]:
df1.loc[df1.offline == True, 'Status'] = 'offline'


In [11]:
df1.loc[df1.online == True, 'Status'] = 'online'

In [12]:
df1['Status'].value_counts()

offline    490351
online     205331
Name: Status, dtype: int64

In [13]:
df1['provider_id'].unique().shape

(14915,)

In [14]:
df2=df1[df1.time > datetime.time(datetime.strptime('1900-01-01 08:00:00','%Y-%m-%d %H:%M:%S'))]
df3=df1[df1.time <= datetime.time(datetime.strptime('1900-01-01 08:00:00','%Y-%m-%d %H:%M:%S'))]

In [15]:
df4=df3.groupby(['provider_id','date'])['time'].max()
df4.shape

(30274,)

In [19]:
df5=df4.to_frame().reset_index()
df5=df5.drop_duplicates(subset=None, keep=False)

In [20]:
df6= pd.merge(left=df1,right=df5, left_on=['provider_id','date','time'], right_on=['provider_id','date','time'])


In [21]:
frames = [df2, df6]

In [22]:
df7 = pd.concat(frames)

In [24]:
def get_status(df7): 
    if df7['time'] <=datetime.time(datetime.strptime('1900-01-01 08:00:00','%Y-%m-%d %H:%M:%S')): 
        return datetime.time(datetime.strptime('1900-01-01 08:00:00','%Y-%m-%d %H:%M:%S')) 
    elif df7['time'] >datetime.time(datetime.strptime('1900-01-01 08:00:00','%Y-%m-%d %H:%M:%S')): 
        return df7['time']

df7['New_time'] = df7.apply(get_status, axis = 1)

In [25]:
df8 = df7.sort_values(['provider_id', 'date', "New_time"])


In [26]:
df8["offset_status"] = df8["Status"].shift(1)

In [27]:
df8["offset_time"] = df8["New_time"].shift(1)


In [28]:
df8["offset_date"] = df8["date"].shift(1)


In [29]:
df8["offset_pid"] = df8["provider_id"].shift(1)


In [30]:
def  online_secs (x):
    td_diff = timedelta(seconds=0)
    if x["provider_id"] == x["offset_pid"]:
        if x["date"] == x["offset_date"]:
            if x["Status"] == x["offset_status"] and x["Status"] == "offline":
                td_diff = timedelta(seconds=0) 
            elif x["Status"] == x["offset_status"] and x["Status"] == "online":
                strt_time = datetime.combine(x["offset_date"], x["offset_time"])
                end_time = datetime.combine(x["date"], x["New_time"])
                td_diff = end_time-strt_time
            elif x["Status"] != x["offset_status"] and x["Status"] == "online" and x["offset_status"] == "offline" :
                td_diff = timedelta(seconds=0) 
            elif x["Status"] != x["offset_status"] and x["Status"] == "offline" and x["offset_status"] == "online" :
                strt_time = datetime.combine(x["offset_date"], x["offset_time"])
                end_time = datetime.combine(x["date"], x["New_time"])
                td_diff = end_time-strt_time
    
    return td_diff.total_seconds()

In [31]:
df8["seconds_online"] = df8.apply(online_secs, axis = 1)


In [32]:
df8["start_time"] = df8.New_time.apply(lambda x:int(str(x)[:2]))
df8["end_time"] = df8.New_time.apply(lambda x:int(str(x)[:2])+1)

In [33]:
df_final = df8.groupby(["provider_id", "date", "start_time", "end_time"])["seconds_online"].sum()

In [34]:
df_final1 = df_final.to_frame().reset_index()

In [35]:
df_final1[df_final1["seconds_online"] > 0.0]

,provider_id,date,start_time,end_time,seconds_online
261,0037dfffff8b03bbdf366a263735e84b,2017-09-04,8,9,1760.0
263,0037dfffff8b03bbdf366a263735e84b,2017-09-04,12,13,11842.0
264,0037dfffff8b03bbdf366a263735e84b,2017-09-04,17,18,140.0
265,0037dfffff8b03bbdf366a263735e84b,2017-09-05,9,10,1748.0
267,0037dfffff8b03bbdf366a263735e84b,2017-09-05,13,14,7285.0
268,0037dfffff8b03bbdf366a263735e84b,2017-09-05,15,16,8021.0
272,0037dfffff8b03bbdf366a263735e84b,2017-09-06,11,12,4913.0
273,0037dfffff8b03bbdf366a263735e84b,2017-09-06,12,13,3577.0
275,0037dfffff8b03bbdf366a263735e84b,2017-09-06,15,16,915.0
276,0037dfffff8b03bbdf366a263735e84b,2017-09-06,16,17,3958.0
